<a href="https://colab.research.google.com/github/yeonghun00/stock_public/blob/main/issue_analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [33]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np
import datetime
import html
import re

# 특징주
def get_search(keyword='%C6%AF%C2%A1%C1%D6', start_date='1', end_date='1'):
  start_date = str(datetime.datetime.now().date().strftime("%Y-%m-%d")) if start_date == '1' else start_date
  end_date = str(datetime.datetime.now().date().strftime("%Y-%m-%d")) if end_date == '1' else end_date

  li = []
  for page in range(1,50000):
    url = 'https://finance.naver.com/news/news_search.nhn?rcdate=&q=' + keyword +\
    '&sm=title.basic&pd=4&stDateStart=' + start_date + '&stDateEnd=' + end_date + '&page=' + str(page)
    result = requests.get(url, headers = {'User-Agent' : 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_14_6) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/85.0.4183.121 Safari/537.36'})
    bs_obj = BeautifulSoup(result.content, "html.parser")
    text = [x.get_text() for x in bs_obj.find_all(re.compile('d*'), {'class':"articleSubject"})] 
    if not text:
      break
    li += text

  return li

In [34]:
get_search('%C6%AF%C2%A1%C1%D6','2021-06-01', '2021-06-12')

https://finance.naver.com/news/news_search.nhn?rcdate=&q=%C6%AF%C2%A1%C1%D6&sm=title.basic&pd=4&stDateStart=2021-06-01&stDateEnd=2021-06-12&page=26


["\n[특징주] 현대차·SK·포스코·효성 '수소 연합' 결성에 수소株 강세\n",
 '\n[특징주] 엘비루셈, 코스닥 상장 첫날 하락…공모가보다는 8.6% 높아(종합)\n',
 '\n[특징주] 비트컴퓨터, 정부 비대면 진료 규제 개선에 관련 사업 부각\n',
 '\n[특징주] 로지시스, 이주열 총재 "하반기 CBDC 모의실험 착수" 발언에 상승세\n',
 "\n[특징주]신풍제약, 코로나 종식 백신에 '먹는 치료제'도 필수…정부도 확보 노력\n",
 '\n[특징주] 넥스턴바이오, 당뇨 1위 글로벌 기업과 협력… 8%↑\n',
 '\n[특징주] 네오티스, 수소차 테마 강세에 24%↑\n',
 "\n[특징주] 가온미디어, KT에 기가지니3 공급 소식에 '급등'\n",
 '\n[특징주] 평화산업, 미래차 지원 기대감에 강세… 52주 신고가\n',
 "\n[특징주] 넥스트사이언스, '나노젠' 코로나 백신 베트남 임상3상 최종 승인…대규...\n",
 '\n[특징주]헬릭스미스, 주주명부 폐쇄 다가오는 데 주가 상승…표대결 준비하나\n',
 "\n[특징주]이준석 국민의힘 당대표 당선…'이준석 테마주' 강세\n",
 '\n[특징주]기업 분할에 SKT ‘껑충’ 만도 ‘강보합’\n',
 '\n[특징주] 이준석 국민의힘 당대표 당선…삼보산업 등 관련株 들썩\n',
 '\n[특징주] 포스코엠텍, 현대차·포스코·SK·효성 수소동맹 수혜주 부각\n',
 '\n[특징주]LG헬로비전, 3일 연속 상승세\n',
 '\n[특징주] 바이오톡스텍, 정부 연매출 1조 블록버스터 신약 개발…임상인프라 지원 ...\n',
 "\n[특징주]이스타항공 인수전 참여 하림...3일째 급등 '52주 신고가'\n",
 '\n[특징주] 솔루스첨단소재2우B, 10%대 급등\n',
 '\n[특징주]엘티씨, 3000조 수소시장 K수소 동맹 구축에 국내 유일 SOFC기술 ...\n',
 "\n[특징주]나노스, 쌍방울 이스타항공 인수 기대감에 '강세'\n",
 "\n[특징주] 뉴인텍, 41兆 투자 'K-수소동맹' 참여 